# Optimal control example

In [2]:
from pyomo.environ import *
from pyomo.dae import *
import matplotlib.pyplot as plt

In [3]:
m = ConcreteModel()

In [4]:
m.t = ContinuousSet(bounds=(0, 1))

In [5]:
m.x1 = Var(m.t, bounds=(0, 1))
m.x2 = Var(m.t, bounds=(0, 1))
m.u = Var(m.t, initialize=0)

In [6]:
m.x1dot = DerivativeVar(m.x1)
m.x2dot = DerivativeVar(m.x2)

In [7]:
m.obj = Objective(expr=m.x2[1])

In [8]:
def _x1dot(mod, i):
    if i == 0:
        return Constraint.Skip
    else:
        return mod.x1dot[i] == mod.u[i]

In [9]:
m.x1dot_con = Constraint(m.t, rule=_x1dot)

In [10]:
def _x2dot(mod, i):
    if i == 0:
        return Constraint.Skip
    else:
        return mod.x2dot[i] == mod.x1[i] ** 2 + mod.u[i] ** 2

In [11]:
m.x2dot_con = Constraint(m.t, rule=_x2dot)

In [12]:
m.init_conditions = ConstraintList()

In [13]:
m.init_conditions.add(m.x1[0] == 1)  #: initial condition for x1
m.init_conditions.add(m.x2[0] == 0)  #: initial condition for x2

## Transformation

In [14]:
discretizer = TransformationFactory("dae.collocation")

In [15]:
discretizer.apply_to(m, nfe=20, ncp=3, scheme="LAGRANGE-RADAU")

In [16]:
discretizer.reduce_collocation_points(m, var=m.u, ncp=1, contset=m.t)

In [17]:
solver = SolverFactory("ipopt")

In [18]:
results = solver.solve(m, tee=True)

    ipopt


ApplicationError: No executable found for solver 'ipopt'

In [ ]:
x1 = []
x2 = []
u = []
t = []

for i in sorted(m.t):
    t.append(i)
    x1.append(value(m.x1[i]))
    x2.append(value(m.x2[i]))
    u.append(value(m.u[i]))

plt.plot(t,x1)
plt.plot(t,x2)
plt.show()

plt.plot(t,u)
plt.show()